In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
# Load the MNIST dataset
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, parser='auto', as_frame=False)

# Create a plot with 2 rows and 10 columns for 20 digits
fig, axes = plt.subplots(2, 10, figsize=(12, 3))
axes = axes.ravel()

# Get and reshape first 20 examples from 784 pixels to 28x28 images
X_subset = X[:20].reshape(-1, 28, 28)
y_subset = y[:20]

# Plot each digit
for idx, (ax, img, label) in enumerate(zip(axes, X_subset, y_subset)):
    ax.imshow(img, cmap='binary')
    ax.set_title(label)
    ax.axis('off')  # Hide axes

plt.tight_layout()
plt.show()

**Summary:**
* Used fetch_openml to get the MNIST handwritten digit dataset [1], [2]
* Created a visualization grid showing 20 sample digits
* Added labels and removed axes for cleaner visualization

**Reasoning:**
* Removed axis ticks since they don't provide meaningful information for images [2]
* Used binary colormap since handwritten digits are normally black and white [2]

**Results Analysis:**
* The dataset holds 784 features (28x28 pixels)
* There are 10 classes representing digits 0-9
* The images show clear digits with variation in writing styles

### 2. Train a Logistic Regression classifier on this data, and report on your findings.
    
1. Tune your hyperparameters to ensure *sparse* weight vectors and high accuracy.
2. Visualise the classification vector for each class.

In [ ]:
# Split data into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train logistic regression
log_reg = LogisticRegression(
    penalty='l1',     # Use L1 regularization for sparse weights
    solver='saga',    # Solver that works well with L1
    tol=0.001,        # Allows for easier convergance
    C=0.05,           # Strength of regularization
    random_state=42,  # For reproducability
    max_iter=200,     # Maximum number of iterations
    n_jobs=-1         # Use all available CPU cores
)

# Fit model and make predictions
log_reg.fit(X_train_scaled, y_train)
y_pred = log_reg.predict(X_test_scaled)

# Print model performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Calculate sparsity (percentage of zero coefficients)
sparsity = np.mean(log_reg.coef_ == 0) * 100
print(f'\nSparsity: {sparsity:.1f}% of coefficients are zero')

**Summary:**
* Split data using train_test_split with:
  * 80-20 train-test ratio
  * random_state=42 for reproducibility
* Standardized features using StandardScaler
* Configured LogisticRegression with:
  * penalty='l1'
  * solver='saga'
  * C=0.05
  * tol=0.001
  * random_state = 42
  * max_iter=200
  * n_jobs=-1
* Printed accuracy and classification report

**Reasoning:**
* test_size=0.2
  * Ensures enough training data [3]
  * Standard split ratio for large datasets [3]
* penalty='l1' [1]. [4], [5]
  * Creates sparse weight vectors
* solver='saga' [1], [4], [5], [6]
  * Optimized for L1 regularization
  * Works well with many features
* C=0.05 [1], [4], [5], [6]
  * Strong regularization for sparsity
  * Found through trial and error
* tol=0.001 [1], [4], [5], [6]
  * Strict enough for convergence
  * Not too strict to slow training
* random_state=42 [4], [7]
  * Ensures reproducability [4]
* max_iter=200 [4], [6]
  * Allows model to converge
  * Despite warning, achieved good results
* n_jobs=-1 [4]
  * Uses all CPU cores
  * Speeds up training
  * No downside on personal machine

**Results Analysis:**
* Reached 91.79% on test set
* Consistently high performance across digits
* Some digits harder to classify than others
  * Best: Digit 0 (96% precision)
  * Worst: Digit 5 (88% precision)
  * Most balanced: Digit 6 (94% precision, 95% recall)
* Sparsity
  * 51.8% of coefficients are zero
  * Shows effective feature selection
  * Keeps high accuracy despite simplification

In [ ]:
# Visualize classification vectors for each digit
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.ravel()

for digit in range(10):
    # Get coefficients for current digit and reshape to image dimensions
    coef = log_reg.coef_[digit].reshape(28, 28)
    axes[digit].imshow(coef, cmap='RdBu')
    axes[digit].axis('off')
    axes[digit].set_title(f'Digit {digit}')

plt.show()

**Summary:**
* Created a 2x5 grid of subplots
* Reshaped classification vectors to 28x28 images
* Displayed using RdBlu colormap

**Reasoning:**
* Selected RdBu colormap [1]
  * Red shows positive weights (features supporting classification)
  * Blue shows negative weights (features opposing classification)
  * White indicates neutral or zero weights
* Removed axes [1]
  * Pixel coordinates aren't meaningful
  * Makes visualization cleaner
  * Matches style of original digit display

**Results Analysis:**
* Vague patterns are interpretable
  * Digit 0 shows circular pattern
  * Digit 8 shows overlapping circles
* Sparsity is visible
  * Large white areas in some visualizations
  * Sharp transitions between important/unimportant regions


### 3. Use PCA to reduce the dimensionality of your training data.
    
1. Determine the number of components necessary to explain 80\% of the variance
2. Plot the explained variance by number of components.
3. Visualise the 20 principal components' loadings
4. Plot the two principal components for your data using a scatterplot, colouring by class. What can you say about this plot?
5. Visualise the first 20 digits, *generated from their lower-dimensional representation*.

In [ ]:
# Scale the data for PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Find number of components for 80% variance
pca = PCA()
pca.fit(X_scaled)
cumsum = np.cumsum(pca.explained_variance_ratio_)
n_components_80 = np.argmax(cumsum >= 0.8) + 1
print(f"Components needed for 80% variance: {n_components_80}")

**Summary:**
* Scaled features using StandardScaler
* Applied PCA without dimension reduction
* Computed cumulative explained variance
* Found minimum components for 80% variance threshold

**Reasoning:**
* Applied StandardScaler
  * PCA is sensitive to feature scales [8]
* Used full PCA initially to ensure proper component selection [9], [10]
* Calculated cumulative variance to choose optimal components [9]

**Results Analysis:**
* Components needed: 150
* 784 dimensions originally
* ~81% reduction in features

In [ ]:
# Plot explained variance
plt.figure(figsize=(10, 4))
plt.plot(cumsum)
plt.axhline(y=0.8, color='r', linestyle='--')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA Explained Variance')
plt.grid(True)
plt.show()

**Summary:**
* Plotted cumulative variance against number of components
* Added a horizontal line at 80% threshold

**Reasoning:**
* Used line plot because it shows continuous build up of variance [9]
* Added red dashed line at 0.8 to highlight the target threshold

**Results Analysis:**
* Sharp initial rise (first ~50 components)
  * First few components capture most variance
* Gradual flattening after elbow
  * Diminishing returns after ~150 components
* 80% threshold intersection occurs at 150 components


In [ ]:
# Visualize first 20 principal components
fig, axes = plt.subplots(4, 5, figsize=(15, 12))
axes = axes.ravel()

for idx in range(20):
    component = pca.components_[idx].reshape(28, 28)
    axes[idx].imshow(component, cmap='RdBu')
    axes[idx].axis('off')
    axes[idx].set_title(f'PC {idx+1}')

plt.show()

**Summary:**
* Created a 4x5 grid of subplots
* Reshaped components to 28x28 images
* Displayed using RdBu colormap

**Reasoning:**
* Selected RdBu colormap [2], [10], [11]
  * Red shows positive contributions
  * Blue shows negative contributions
  * White indicates neutral areas
  * Matches the logistic regression visualizations
* Removed axes
  * Makes visualization cleaner [2]
  * Focuses attention on patterns

**Results Analysis:**
* First components show basic digit structures and large-scale patterns
* Later components capture more complex patterns



In [ ]:
# Show data in 2D using first two components
pca_2d = PCA(n_components=2)
X_pca = pca_2d.fit_transform(X_scaled)

plt.figure(figsize=(10, 8))
plt.scatter(X_pca[:, 0], X_pca[:, 1],
           c=y.astype(int),
           cmap='tab10',
           alpha=0.5)
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.colorbar(label='Digit')
plt.show()

**Summary:**
* Reduced data to 2 dimensions using PCA [10]
* Created a scatter plot of all samples [11]
* Colored points by digit class
* Added axis labels and colorbar

**Reasoning:**
* Selected tab10 colormap
  * Designed for categorical data
  * Each digit gets distinct color
* alpha=0.5
  * Show point density
  * Display overlapping regions

**Results Analysis:**
  * Imperfect separation
    * Makes sense given high dimension reduction
  * Complex digits look clustered (3, 4, 5, 8)
  * Distinct digits are spread further apart (7, 1, 0)

In [ ]:
# Show reconstructed digits using reduced dimensions
pca = PCA(n_components=n_components_80)
X_reduced = pca.fit_transform(X_scaled)
X_reconstructed = pca.inverse_transform(X_reduced)
X_reconstructed = scaler.inverse_transform(X_reconstructed)

# Plot first 20 reconstructed digits
fig, axes = plt.subplots(4, 5, figsize=(12, 10))
axes = axes.ravel()

for i in range(20):
    axes[i].imshow(X_reconstructed[i].reshape(28, 28), cmap='binary')
    axes[i].axis('off')
    axes[i].set_title(f'Digit {y[i]}')

plt.show()

**Summary:**
* Reduced data to 80% variance components [9], [10], [12]
* Reconstructed digits from reduced space [10]
* Transformed back to original scale
* Displayed first 20 reconstructed digits [2]

**Reasoning:**
* Used n_components_80 to match the earlier variance analysis
* Chose binary colormap because it matches original MNIST style

**Results Analysis:**
  * Main digit shapes preserved
  * Slight blurring in complex regions
  * Shows 80% variance is sufficient to perserve information

### 4. Generate a noisy copy of your data by adding random normal noise to the digits **with a scale that doesn't completely destroy the signal**. This is, the resulting images noise should be apparent, but the numbers should still be understandable.
    
1. Visualise the first 20 digits from the noisy dataset.
2. Filter the noise by fitting a PCA explaining **a sufficient proportion** of the variance, and then transforming the noisy dataset. Figuring out this proportion is part of the challenge.
3. Visualise the first 20 digits of the de-noised dataset.

In [ ]:
# Add random noise to images
noise_level = 60
X_noisy = X + np.random.normal(0, noise_level, X.shape)

# Show noisy digits
fig, axes = plt.subplots(4, 5, figsize=(12, 10))
axes = axes.ravel()

for i in range(20):
    axes[i].imshow(X_noisy[i].reshape(28, 28), cmap='binary')
    axes[i].axis('off')
    axes[i].set_title(f'Digit {y[i]}')

plt.show()

**Summary:**
* Added noise with scale=60 [13]
* Maintained original image dimensions
* Visualized first 20 noisy digits [2]

**Reasoning:**
* noise_level=60
  * Make noise clearly visible
  * Keep digits recognizable
  * Create challenging denoising task
* Used binary colormap to match original MNIST style

**Results Analysis:**
  * Clear random variations added
  * Digit shapes still visible
  * Varying impact on different images

In [ ]:
# Scale and denoise using PCA
scaler = StandardScaler()
X_noisy_scaled = scaler.fit_transform(X_noisy)

# PCA with 90% variance kept
pca = PCA(n_components=0.90)
X_denoised = pca.inverse_transform(pca.fit_transform(X_noisy_scaled))
X_denoised = scaler.inverse_transform(X_denoised)

print(f"Using {pca.n_components_} components for denoising")

**Summary:**
* Standardized the noisy images
* Applyied PCA with 90% variance threshold [9], [10]
* Reconstructed denoised images [10]

**Reasoning:**
* 90% variance threshold [9]
  * Higher than 80% used for compression
  * Better separates noise from signal

**Results Analysis:**
  * Higher than 150 components used for compression
  * Appropriate due to noise presence

In [ ]:
# Show denoised digits
fig, axes = plt.subplots(4, 5, figsize=(12, 10))
axes = axes.ravel()

for i in range(20):
    axes[i].imshow(X_denoised[i].reshape(28, 28), cmap='binary')
    axes[i].axis('off')
    axes[i].set_title(f'Digit {y[i]}')

plt.show()

**Summary:**
* Created 4x5 grid of denoised digits
* Reshaped each to 28x28 images
* Used binary colormap display

**Reasoning:**
* Kept binary colormap because it matches original MNIST style
* Removed axes [2]
  * Keep visualization clean
  * Focus on denoising quality
  * Maintain consistent style

**Results Analysis:**
  * Significant noise removal
  * Cleaner digit boundaries
  * Some loss compared to originals

#### Citations
[1] Scikit Learn. "MNIST classification using multinomial logistic + L1." scikit-learn.org. Accessed: Dec. 1, 2024. [Online.] Available: https://scikit-learn.org/stable/auto_examples/linear_model/plot_sparse_logistic_regression_mnist.html

[2] V. Ojha. (2024). Image Classification DNNs [Google Colab notebooks]. Available: https://ncl.instructure.com/courses/55046/modules/items/3539140

[3] V. Gonzalez-Zelaya. (2024). Introduction to Machine Learning / Regression Problems [PowerPoint slides]. Available: https://ncl.instructure.com/courses/55046/files/8847813?module_item_id=3517175

[4] Scikit Learn. "LogisticRegression." scikit-learn.org. Accessed: Dec. 2, 2024. [Online.] Available: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html


[5] V. Ojha. (2024). Convolutional Neural Network [PowerPoint slides]. Available: https://ncl.instructure.com/courses/55046/files/8928967?module_item_id=3535263

[6] A. Mani. "MNIST digits classification using Logistic regression in Scikit-Learn." atmamani.github.io. Accessed: Dec. 1, 2024. [Online.] Available: https://atmamani.github.io/projects/ml/mnist-digits-classification-using-logistic-regression-scikit-learn/

[7] V. Gonzalez-Zelaya. (2024). Practical 2 [Google Colab notebook]. Available: https://colab.research.google.com/drive/1_8ODB5_nC0p9mh9wl3wGY3xxD5L_rLV5?usp=sharing

[8] V. Gonzalez-Zelaya. (2024). Unsupervised Learning [PowerPoint slides]. Available: https://ncl.instructure.com/courses/55046/files/8884895?module_item_id=3528211

[9] V. Gonzalez-Zelaya. (2024). Principal Components Analysis (PCA) [Google Colab notebook]. Available: https://colab.research.google.com/drive/1rwHmzIA18PTudDy_HzIDQ0HKbkmk_wbZ?usp=sharing


[10] Scikit Learn. "PCA." scikit-learn.org. Accessed: Dec. 2, 2024. [Online.] Available: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

[11] A. Tam. "Principal Component Analysis for Visualization." machinelearningmastery.com. Accessed: Nov. 28, 2024. [Online.] Available: https://machinelearningmastery.com/principal-component-analysis-for-visualization/


[12] Stack Overflow. "PCA projection and reconstruction in scikit-learn." stackoverflow.com. Accessed: Dec. 3, 2024. [Online.] Available: https://stackoverflow.com/questions/36566844/pca-projection-and-reconstruction-in-scikit-learn

[13] M. Somanna. "Guide to Adding Noise to Synthetic Data using Python and Numpy." medium.com. Accessed: Dec. 1, 2024. [Online.] Available: https://medium.com/@ms_somanna/guide-to-adding-noise-to-your-data-using-python-and-numpy-c8be815df524